# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [63]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [64]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,viedma,-40.8135,-62.9967,23.77,18,0,3.47,AR,1699543633
1,1,port-aux-francais,-49.3500,70.2167,6.05,94,100,12.81,TF,1699543186
2,2,saipan,15.1355,145.7010,28.39,78,20,5.14,MP,1699543633
3,3,waitangi,-43.9535,-176.5597,8.29,74,95,13.14,NZ,1699543187
4,4,pundong,-7.9522,110.3486,25.54,87,100,0.86,ID,1699543634


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [65]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = 'Humidity',
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.6,
    color = "City",
    alpha=0.5
)

# Display the map plot
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [66]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.dropna(inplace=True)

# Drop any rows with null values
filtered_cities = city_data_df[(city_data_df['Max Temp'] > 21) & 
                                (city_data_df['Max Temp'] <= 27) & 
                                (city_data_df['Wind Speed'] < 4.5) & 
                                (city_data_df['Cloudiness'] == 0)]


# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,viedma,-40.8135,-62.9967,23.77,18,0,3.47,AR,1699543633
29,29,khark,29.2614,50.3306,26.01,65,0,2.06,IR,1699543639
129,129,urangan,-25.3000,152.9000,21.12,69,0,3.07,AU,1699543662
159,159,port alfred,-33.5906,26.8910,22.32,73,0,0.45,ZA,1699543314
318,318,kailua-kona,19.6406,-155.9956,23.98,85,0,4.12,US,1699543233
364,364,palompon,11.0508,124.3843,26.57,84,0,2.58,PH,1699543721
386,386,dewas,22.9667,76.0667,24.10,41,0,2.83,IN,1699543725
388,388,mizdah,31.4451,12.9801,24.86,39,0,4.14,LY,1699543725
389,389,saint-pierre,-21.3393,55.4781,25.82,73,0,1.54,RE,1699543303
424,424,galliano,29.4422,-90.2992,26.00,73,0,3.09,US,1699543733


### Step 3: Create a new DataFrame called `hotel_df`.

In [67]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel
0,viedma,AR,-40.8135,-62.9967,18,
29,khark,IR,29.2614,50.3306,65,
129,urangan,AU,-25.3000,152.9000,69,
159,port alfred,ZA,-33.5906,26.8910,73,
318,kailua-kona,US,19.6406,-155.9956,85,
364,palompon,PH,11.0508,124.3843,84,
386,dewas,IN,22.9667,76.0667,41,
388,mizdah,LY,31.4451,12.9801,39,
389,saint-pierre,RE,-21.3393,55.4781,73,
424,galliano,US,29.4422,-90.2992,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [69]:
# Set parameters to search for a hotel
radius = 10000
params = {
        'limit' : 1,
        'categories' : 'accommodation.hotel',
        'apiKey': geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat =row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel']}")

# Display sample data
hotel_df

Starting hotel search
viedma - nearest hotel: Níjar
khark - nearest hotel: No hotel found
urangan - nearest hotel: Ramada
port alfred - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
palompon - nearest hotel: No hotel found
dewas - nearest hotel: No hotel found
mizdah - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
arar - nearest hotel: دار أوتيل Dar Otel
riyadh - nearest hotel: White Palace Hotel
shush - nearest hotel: هتل آپادانا
holualoa - nearest hotel: Kona Hotel


,City,Country,Lat,Lng,Humidity,Hotel
0,viedma,AR,-40.8135,-62.9967,18,Níjar
29,khark,IR,29.2614,50.3306,65,No hotel found
129,urangan,AU,-25.3000,152.9000,69,Ramada
159,port alfred,ZA,-33.5906,26.8910,73,No hotel found
318,kailua-kona,US,19.6406,-155.9956,85,Kona Seaside Hotel
364,palompon,PH,11.0508,124.3843,84,No hotel found
386,dewas,IN,22.9667,76.0667,41,No hotel found
388,mizdah,LY,31.4451,12.9801,39,No hotel found
389,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
424,galliano,US,29.4422,-90.2992,73,Fairfield Inn & Suites by Marriott Cut Off-Gal...


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [84]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size='Humidity',
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    scale=0.7,
    hover_cols=['Country','Hotel'],
    color='City',
    alpha=0.5
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel)